In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import geom
from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import LogisticRegression as LogR
from sklearn.tree import DecisionTreeRegressor as DT
np.random.seed(10)

# Problem Statement

### We want to predict when people might have surgery

## Challenges



### Suppose the probabiltiy ($p$) of an event occurring on a given day is not a fixed value, but instead is a variable the depends on one or more risk factors.

We have a population of people aged 25-75, where probability of getting surgery increases with age according to $\large p = \frac {1}{1+e^{(-a-b\cdot \text {age})}}$

Where $a$ is the intercept of the log-odds (e.g. baseline risk) and $b$ is the hazard associated with each unit increase of age.

In [ ]:
n = 5000
df = pd.DataFrame()
df['age'] = np.random.randint(25, 76, size=n)

In [ ]:
df

In [ ]:
a=-10.5
b=0.05
timeout = 365*3

In [ ]:
prob_values = 1/(1+np.exp(-a-b*df['age']))

In [ ]:
plt.scatter(df['age'],1/(365*prob_values))
plt.xlabel('Age')
plt.ylabel('Years until surgery');

### We would like to observe these people until they get surgery, but in some cases, that might be over 150 years.

We have a cut-off at 3 years.

In [ ]:
days = geom(prob_values).rvs()
df['observation'] = np.where(days <= timeout, days, np.nan)
df

In [ ]:
(df['observation'].isna()).mean()

In [ ]:
days.max()/365

In [ ]:
plt.scatter(df['age'],days/365, alpha=.1)
plt.axhline(timeout/365, color='red')
plt.xlabel('Age')
plt.ylabel('Years to Surgery');

In [ ]:
plt.scatter(df['age'], df['observation'], alpha = .1)
plt.xlabel('age')
plt.ylabel('Days to Surgery');

In [ ]:
S = LogR().fit(df[['age']],df['observation'].isna()).predict_proba(df[['age']])[:,0]
plt.scatter(df['age'], S)
plt.xlabel('Age')
plt.ylabel('Observed Surgeries')
plt.title('Surgery Observations by Age');

$$\text {Imputed Days to Surgery} = timeout + \frac {1}{1-\left (1-S(age)\right )^{\left(\frac {1}{timeout}\right)}}$$

In [ ]:
df['imputed_days'] = np.where(df['observation'].isna(), timeout + (1/(1-(1-S)**(1/timeout))), df['observation'])
df

In [ ]:
plt.scatter(df['age'], df['imputed_days'], alpha = .05);

In [ ]:
df['probability'] = 1/DT().fit(df[['age']], df['imputed_days']).predict(df[['age']])
df

In [ ]:
plt.scatter(df['age'],1/prob_values, label = 'Unobserved true values');
plt.scatter(df['age'], 1/df['probability'], label = 'Estimated days')
plt.scatter(df['age'], df['imputed_days'], alpha = .05, label = 'Observed & Imputed Values')
plt.legend()

In [ ]:
log_odds = np.log(df['probability']/(1-df['probability']))
log_odds

In [ ]:
lr = LR().fit(df[['age']], log_odds)

In [ ]:
plt.scatter(df['age'], log_odds, s=10)
plt.plot(df['age'], df['age']*b+a, color = 'red')
plt.plot(df['age'], lr.predict(df[['age']]), color = 'green');

In [ ]:
lr.intercept_, a

In [ ]:
lr.coef_, b

# Results

### What is the probability that a member will have surgery in 6-18 months?

In [ ]:
df['target']= df['probability'].apply(lambda p:geom(p).cdf(18*30)-geom(p).cdf(6*30))
df

In [ ]:
plt.scatter(df['age'], df['target'])
ages = np.linspace(20, 80)
plt.plot(ages, np.exp(LR().fit(np.log(df[['age']]),
                                       np.log(df['target'])
                                      )
                              .predict(np.log(ages.reshape(-1,1)))),
                            color = 'red');


# What if we want to predict outside of our observation window?

#### How many people in this population would have surgery within 3-5 years?

In [ ]:
((days>(3*365))&(days<(5*365))*1).sum()

#### How many were *observed* to have surgery in 3-5 years?

In [ ]:
((df['observation']>(3*365))&(df['observation']<(5*365))*1).sum()

#### How many people do we *predict* will have surgery in 3-5 years?

In [ ]:
df['probability'].apply(lambda p:geom(p).cdf(5*365)-geom(p).cdf(3*365)).sum()

# Multi-variate Example

In [ ]:
features = 10
df = pd.DataFrame(data = np.random.random(size = (20000, features)))
w = np.random.random(features)-.5

In [ ]:
intercept = -6

In [ ]:
w

In [ ]:
df['p'] = 1/(1+np.exp(-intercept-df.apply(lambda x:x.dot(w),axis=1)))

In [ ]:
df

In [ ]:
plt.hist(df['p']);

In [ ]:
df['N'] = geom(df['p']).rvs()
df

In [ ]:
timeout = np.percentile(df['N'], 55)
timeout

In [ ]:
plt.scatter(df[0], df['N'], alpha=.1)
plt.axhline(timeout, color='red')
plt.xlabel('Risk Factor 0')
plt.ylabel('Days to Surgery (Observed and Unobserved)');

In [ ]:
df['observation'] = np.where(df['N']<=timeout, df['N'], np.nan)

In [ ]:
plt.scatter(df[0], df['observation'], alpha = .02)
plt.xlabel('Risk Factor 0')
plt.ylabel('Days to Surgery (Observed Only)');

In [ ]:
df['S'] = np.where(df['observation'].isna(),
                   LogR().fit(df.iloc[:,:features],df['observation'].isna())\
.predict_proba(df.iloc[:,:features])[:,0],
                   np.nan)
df

In [ ]:
df['Λ'] = timeout + 1/(1-(1-df['S'])**(1/timeout))
df

In [ ]:
df['x'] = np.where(df['observation'].isna(), df['Λ'], df['observation'])
df

In [ ]:
from sklearn.ensemble import RandomForestRegressor as RFC
from sklearn.neighbors import KNeighborsRegressor as KNN

In [ ]:
df['x_pred'] = KNN(20).fit(df.iloc[:,:features],
                        df['x']).predict(df.iloc[:,:features])

In [ ]:
df['log_odds'] = LR().fit(df.iloc[:,:features],
np.log((1/df['x_pred'])/(1-(1/df['x_pred'])))).predict(df.iloc[:,:features])
df

In [ ]:
plt.scatter(df['p'], 1/(1+1/np.exp(df['log_odds'])), alpha=.25)
plt.xticks(rotation=45)

plt.plot([df['p'].min(), df['p'].max()],[df['p'].min(), df['p'].max()],  color = 'red')

plt.xlabel('Ideal Probability')
plt.ylabel('Estimated Probability');

## Evaluation of Risk Factors

We can evaluate how much each feature contributes to a person's risk from the observed data by fitting an equation between the features and the log odds.

In [ ]:
lr = LR().fit(df.iloc[:,:features], df['log_odds'])

We can compare these empirical values to the values used to create the dataset:

In [ ]:
lr.intercept_, intercept

In [ ]:
lr.coef_

In [ ]:
w

In [ ]:
plt.scatter(w, lr.coef_)
plt.plot([w.min(), w.max()], [w.min(),w.max()], color = 'red')
plt.xlabel('Actual Hazard Coeficients')
plt.ylabel('Measured Hazard Coeficients');